# Cost Basis Calculation
### Variables

1. DATE - date in M D format
2. POSITION - Long if BUY/SELL, Short if SHORT/COVER
3. QUANTITY - Cumulative Quantity, add if BUY/SHORT, minus if SELL/COVER
4. COST - PC * Quantity
5. FULL COST - Cost * 1.01 or cost + 2cents per share (depending on which is higher)
6. PC - Weighted Price, If BUY/SHORT add to weight else same as previous 
7. PFC - Full cost/Quantity

#### Import Libraries

In [1]:
import platform
import pyodbc
import pandas as pd

#### Import Data from Database

In [2]:
def db_connect_str_from_env ():
    if platform.system() == 'Windows':
        driver = 'DRIVER={SQL Server};'
    elif platform.system() == 'Darwin': #MacOS
        if platform.machine() == 'arm64': #M1 chip
            driver = 'DRIVER=/opt/homebrew/lib/libmsodbcsql.18.dylib;'
        else:
            driver = 'DRIVER=/Library/simba/sqlserverodbc/lib/libsqlserverodbc_sbu.dylib;'
    
    return driver + 'SERVER=dlyle.database.windows.net;DATABASE=HSX;UID=student;PWD=Viz(Data);'

DB = db_connect_str_from_env()

In [3]:
SQL = """
    SELECT *
    FROM Trades
    WHERE User_Name in ('rkhoo', 'will_ho', 'lucasee')
"""
data = pd.read_sql(SQL, pyodbc.connect(DB), parse_dates = ['Trade_Date_Time'])

/Users/weida/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [4]:
data.sort_values(['User_Name', 'Security_Symbol'], inplace = True)
data

,Security_Symbol,User_Name,Trade_Date_Time,Action,Price,Quantity,Amount
76,AQUM2,lucasee,2022-08-21 22:03:00,Buy,239.56,463,-112025.44
77,AQUM2,lucasee,2022-09-15 05:27:00,Sell,223.15,463,102285.27
79,ASCLB,lucasee,2022-10-09 03:05:00,Short,14.82,1000,-14968.20
80,ASCLB,lucasee,2022-10-09 03:06:00,Short,14.82,3000,-44904.60
81,ASCLB,lucasee,2022-10-13 17:52:00,Cover,14.48,4000,60060.80
...,...,...,...,...,...,...,...
317,WARPG,will_ho,2022-08-28 23:01:00,Sell,0.09,150000,10500.00
318,WARPG,will_ho,2022-08-28 23:04:00,Short,0.07,150000,-13500.00
319,WARPG,will_ho,2022-08-28 23:06:00,Cover,0.04,150000,12000.00
322,WICK4,will_ho,2022-08-23 01:15:00,Short,150.53,2000,-304070.60


#### Function Definition

In [5]:
# Convert Action to position label
def position_choice(row):
    # check if delist action belongs to long or short
    if row['Action'] == 'Delist':
        if (row['Quantity'] * row['Price'] == abs(row['Amount'])):
            return 'long'
        else:
            return 'short'
    elif row['Action'] == 'Buy' or row['Action'] == 'Sell':
        return 'long'
    else:
        return 'short'

# Add magnitude to quantity to compute cumulative quantity
def signed_quant(row):
    # if open position, just leave quantity as positive
    if row['Action'] == 'Buy' or row['Action'] == 'Short':
        return row['Quantity']
    # if close position then multiply quantity by -1
    elif row['Action'] == 'Cover' or row['Action'] == 'Sell' or row['Action'] == 'Delist': 
        return row['Quantity'] * -1
    else:
        return 'error'
    
# Calculate cost
def cost_calc(row):
    if row['Quantity'] == 0:
        return '-'
    else:
        return row['Pc'] * row['Quantity']
    
# Calculate commission for full cost
def commission_calc(row):
    if row['Quantity'] == 0:
        return '-'
    # define the two ways to calculate commissions
    two_cent_comm = abs(row['Signed_Quantity']) * 0.02
    one_percent_comm = 0.01 * row['cost']
    # compare and return whichever that is higher
    if two_cent_comm > one_percent_comm:
        return (row['cost'] + two_cent_comm)
    else: 
        return row['cost'] + one_percent_comm

# calculate pfc   
def pfc_calc(row):
    if row['Quantity'] == 0:
        return '-'
    else:
        return row['Full Cost'] / row['Quantity']


    

#### Main Program

In [6]:
# remove leading and trailing white spaces from Action
data['Action'] = data.Action.apply(str.strip)
# Create 'Position' Column
data['Position'] = data.apply(position_choice, axis=1)

In [7]:
# Create a column where quantities are signed
data['Signed_Quantity'] = data.apply(signed_quant, axis=1)

In [8]:
# Group values by user, securities and their position
data.sort_values(['User_Name', 'Security_Symbol', 'Position', 'Trade_Date_Time'], inplace = True)

In [9]:
# 3. Turn quantity column into a cumulative one
data['Quantity'] = data.groupby(by = ['User_Name', 'Security_Symbol'])['Signed_Quantity'].transform(lambda x: x.cumsum())


In [10]:
# Reset and allocate a new index for the df (for itertuple below)
data.reset_index(inplace=True)
for i in range(len(data.index)):
    data.loc[i, 'index'] = i
data.set_index('index')

,Security_Symbol,User_Name,Trade_Date_Time,Action,Price,Quantity,Amount,Position,Signed_Quantity
index,,,,,,,,,
0,AQUM2,lucasee,2022-08-21 22:03:00,Buy,239.56,463,-112025.44,long,463
1,AQUM2,lucasee,2022-09-15 05:27:00,Sell,223.15,0,102285.27,long,-463
2,ASCLB,lucasee,2022-10-09 03:05:00,Short,14.82,1000,-14968.20,short,1000
3,ASCLB,lucasee,2022-10-09 03:06:00,Short,14.82,4000,-44904.60,short,3000
4,ASCLB,lucasee,2022-10-13 17:52:00,Cover,14.48,0,60060.80,short,-4000
...,...,...,...,...,...,...,...,...,...
325,WARPG,will_ho,2022-08-28 23:01:00,Sell,0.09,0,10500.00,long,-150000
326,WARPG,will_ho,2022-08-28 23:04:00,Short,0.07,150000,-13500.00,short,150000
327,WARPG,will_ho,2022-08-28 23:06:00,Cover,0.04,0,12000.00,short,-150000


In [11]:
# Calculate Pc (weighted price) for each column 

# Create three tracker variables to track movie symbol, position type and username that we're checking
movie_symb = ''
user = ''
Pos = ''

for row in data.itertuples():
    # if very first row of df
    if row.index == 0:
        movie_symb = data.loc[row.index, 'Security_Symbol']
        user = data.loc[row.index, 'User_Name']
        pos = data.loc[row.index, 'Position']
        data.loc[row.index,'Pc'] = data.loc[row.index,'Price']
    else:
        # find out quantity and pc of previous row
        prev_quant = data.loc[row.index-1, 'Quantity']
        prev_pc = data.loc[row.index-1, 'Pc']
        # if position is closed, skip
        if data.loc[row.index, 'Quantity'] == 0:
            data.loc[row.index,'Pc'] = '-'
        # if not closed then check if security, position and user matches our tracker variable
        elif movie_symb == data.loc[row.index, 'Security_Symbol'] and pos == data.loc[row.index, 'Position'] and user == data.loc[row.index, 'User_Name'] and prev_quant !=0:
             # if opening new position then calc new weighted price
            if (data.loc[row.index, 'Action'] == 'Buy' or data.loc[row.index, 'Action'] == 'Short'):
                 data.loc[row.index,'Pc'] = (data.loc[row.index, 'Price']*(data.loc[row.index, 'Signed_Quantity']/data.loc[row.index, 'Quantity'])) + (prev_pc*(prev_quant/data.loc[row.index, 'Quantity']))
             # else take weighted price as it is
            else :
                data.loc[row.index,'Pc'] = prev_pc
        # if criteria not met then reset tracker variables and pass pc as the price
        else:
            movie_symb = data.loc[row.index, 'Security_Symbol']
            user = data.loc[row.index, 'User_Name']
            pos = data.loc[row.index, 'Position']
            data.loc[row.index,'Pc'] = data.loc[row.index,'Price']
data

,index,Security_Symbol,User_Name,Trade_Date_Time,Action,Price,Quantity,Amount,Position,Signed_Quantity,Pc
0,0,AQUM2,lucasee,2022-08-21 22:03:00,Buy,239.56,463,-112025.44,long,463,239.56
1,1,AQUM2,lucasee,2022-09-15 05:27:00,Sell,223.15,0,102285.27,long,-463,-
2,2,ASCLB,lucasee,2022-10-09 03:05:00,Short,14.82,1000,-14968.20,short,1000,14.82
3,3,ASCLB,lucasee,2022-10-09 03:06:00,Short,14.82,4000,-44904.60,short,3000,14.82
4,4,ASCLB,lucasee,2022-10-13 17:52:00,Cover,14.48,0,60060.80,short,-4000,-
...,...,...,...,...,...,...,...,...,...,...,...
325,325,WARPG,will_ho,2022-08-28 23:01:00,Sell,0.09,0,10500.00,long,-150000,-
326,326,WARPG,will_ho,2022-08-28 23:04:00,Short,0.07,150000,-13500.00,short,150000,0.07
327,327,WARPG,will_ho,2022-08-28 23:06:00,Cover,0.04,0,12000.00,short,-150000,-
328,328,WICK4,will_ho,2022-08-23 01:15:00,Short,150.53,2000,-304070.60,short,2000,150.53


In [12]:
# Add Cost Column
data['cost'] = data.apply(cost_calc, axis = 1)

In [13]:
# Add full cost column
data['Full Cost'] = data.apply(commission_calc, axis=1)

In [14]:
# Calculate PFC
data['PFC'] = data.apply(pfc_calc, axis=1)

In [15]:
# To calculate realised gains 

close_actions = ['Sell', 'Cover', 'Delist']

for row in data.itertuples():
    # if very first row, just set tracker variables
    if row.index == 0:
        movie_symb = data.loc[row.index, 'Security_Symbol']
        user = data.loc[row.index, 'User_Name']
        data.loc[row.index, 'Realised_Gain'] = '-'
    else:
        # Find out realised gain and pfc of previous row
        prev_r_gain = data.loc[row.index-1, 'Realised_Gain']
        prev_pfc = data.loc[row.index-1, 'PFC']
        # if tracker variable matches, proceed with calculations
        if movie_symb == data.loc[row.index, 'Security_Symbol'] and user == data.loc[row.index,'User_Name']:
            # if we are closing a position we have to contribute to realised gains
            if (data.loc[row.index, 'Action'] in close_actions):
                # if previous row does not have realised gains then can just calculate realised gain of current row
                if prev_r_gain == '-':
                # if position is closed out completely, have to use the pfc from the row above to calculate gains
                    if data.loc[row.index, 'Quantity'] == 0:
                        data.loc[row.index, 'Realised_Gain'] = data.loc[row.index,'Amount'] - (abs(data.loc[row.index,'Signed_Quantity'])*prev_pfc)
                    else:
                        data.loc[row.index, 'Realised_Gain'] = data.loc[row.index, 'Amount'] - (abs(data.loc[row.index,'Signed_Quantity'])*data.loc[row.index,'PFC'])
                # previous row has realised gain and position is closed out completely
                elif data.loc[row.index, 'Quantity']== 0:
                    data.loc[row.index, 'Realised_Gain'] = (prev_r_gain + (data.loc[row.index, 'Amount'] - (abs(data.loc[row.index, 'Signed_Quantity']) * prev_pfc)))
                # previous row has realised gain and position is not closed out completely
                else:
                    data.loc[row.index, 'Realised_Gain'] = (prev_r_gain + (data.loc[row.index, 'Amount'] - (abs(data.loc[row.index, 'Signed_Quantity']) * data.loc[row.index, 'PFC'])))
            #if we're are not closing a position, we either have no realised gain or realised gain equals to that of the previous row
            else:                                             
                if prev_r_gain == '-':
                    data.loc[row.index, 'Realised_Gain'] = '-'
                else:
                    data.loc[row.index, 'Realised_Gain'] = prev_r_gain
        # if tracker variable does not match then reset them
        else:
            movie_symb = data.loc[row.index, 'Security_Symbol']
            user = data.loc[row.index, 'User_Name']
            data.loc[row.index, 'Realised_Gain'] = '-'
                
    

In [16]:
# Format number to 3dp
data.round(3)

,index,Security_Symbol,User_Name,Trade_Date_Time,Action,Price,Quantity,Amount,Position,Signed_Quantity,Pc,cost,Full Cost,PFC,Realised_Gain
0,0,AQUM2,lucasee,2022-08-21 22:03:00,Buy,239.56,463,-112025.44,long,463,239.56,110916.28,112025.4428,241.9556,-
1,1,AQUM2,lucasee,2022-09-15 05:27:00,Sell,223.15,0,102285.27,long,-463,-,-,-,-,-9740.1728
2,2,ASCLB,lucasee,2022-10-09 03:05:00,Short,14.82,1000,-14968.20,short,1000,14.82,14820.0,14968.2,14.9682,-
3,3,ASCLB,lucasee,2022-10-09 03:06:00,Short,14.82,4000,-44904.60,short,3000,14.82,59280.0,59872.8,14.9682,-
4,4,ASCLB,lucasee,2022-10-13 17:52:00,Cover,14.48,0,60060.80,short,-4000,-,-,-,-,188.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,325,WARPG,will_ho,2022-08-28 23:01:00,Sell,0.09,0,10500.00,long,-150000,-,-,-,-,1500.0
326,326,WARPG,will_ho,2022-08-28 23:04:00,Short,0.07,150000,-13500.00,short,150000,0.07,10500.0,13500.0,0.09,1500.0
327,327,WARPG,will_ho,2022-08-28 23:06:00,Cover,0.04,0,12000.00,short,-150000,-,-,-,-,-0.0
328,328,WICK4,will_ho,2022-08-23 01:15:00,Short,150.53,2000,-304070.60,short,2000,150.53,301060.0,304070.6,152.0353,-


In [17]:
# Create df with only columns we want
data_final = data[['User_Name','Security_Symbol','Trade_Date_Time', 'Position', 'Quantity','cost', 'Full Cost', 'Pc', 'PFC', 'Realised_Gain']].copy()

In [18]:
# Create another df where only the last row of each row is displayed
data_final_last_row_only = data_final.groupby(['User_Name', 'Security_Symbol']).tail(1)

In [19]:
# Save both files as CSV
data_final.to_csv('Cost_Basis.csv', index = False)
data_final_last_row_only.to_csv('Cost_Basis_final.csv', index = False)